In [ ]:
! pip install pyro-ppl[extras]

In [ ]:
import pyro
import torch
from pyro.infer.autoguide import AutoMultivariateNormal, AutoDelta
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam as PyroAdam

def setup_pyro_model():
    """Complete setup for Pyro model with proper error handling."""
    
    # Clear parameter store
    pyro.clear_param_store()
    
    # Create model
    model = PhysicsInformedPyroModel(
        X_train_tensor.shape[1], 
        y_train_tensor.shape[1],
        feature_names=feature_names,
        wavelengths=wavelengths
    ).to(device)
    
    print("=== Attempting AutoMultivariateNormal ===")
    
    # Try AutoMultivariateNormal first
    guide = AutoMultivariateNormal(model)
    
    # Initialize guide properly
    sample_x = X_train_tensor[:2].to(device)
    sample_y = y_train_tensor[:2].to(device)
    
    # Run model to initialize guide
    with torch.no_grad():
        model(sample_x, sample_y)
    
    # Force guide initialization by calling it
    with torch.no_grad():
        guide(sample_x, sample_y)
    
    guide_params = list(guide.parameters())
    print(f"AutoMultivariateNormal parameters: {len(guide_params)}")
    
    if len(guide_params) > 0:
        # Use Pyro optimizer (required for SVI)
        optimizer = PyroAdam({"lr": config.PYRO_LR})
        svi = SVI(model, guide, optimizer, loss=Trace_ELBO())
        
        print("✓ AutoMultivariateNormal setup successful!")
        return model, guide, svi, "AutoMultivariateNormal"
    
    print("=== Fallback to AutoDelta ===")
    
    # Fallback to AutoDelta
    pyro.clear_param_store()
    guide_delta = AutoDelta(model)
    
    # Initialize AutoDelta
    with torch.no_grad():
        model(sample_x, sample_y)
        guide_delta(sample_x, sample_y)
    
    delta_params = list(guide_delta.parameters())
    print(f"AutoDelta parameters: {len(delta_params)}")
    
    if len(delta_params) > 0:
        optimizer = PyroAdam({"lr": config.PYRO_LR})
        svi = SVI(model, guide_delta, optimizer, loss=Trace_ELBO())
        
        print("✓ AutoDelta setup successful!")
        return model, guide_delta, svi, "AutoDelta"
    
    print("=== Final fallback: Regular PyTorch training ===")
    
    # Final fallback: regular PyTorch optimizer
    model_params = list(model.parameters())
    if len(model_params) > 0:
        torch_optimizer = torch.optim.Adam(model.parameters(), lr=config.PYRO_LR)
        print("✓ Regular PyTorch optimizer created!")
        print("Note: You'll need to use regular PyTorch training loop, not SVI")
        return model, None, torch_optimizer, "PyTorch"
    
    raise ValueError("Could not create any optimizer - no parameters found anywhere!")

# Alternative approach: Force guide parameter creation
def force_guide_initialization(model, guide, sample_x, sample_y):
    """Force guide to create parameters by accessing internal methods."""
    
    # Try to access the guide's internal setup
    try:
        # This should force parameter creation
        guide._setup_prototype(sample_x, sample_y)
        guide._prototype_trace = guide._get_prototype_trace(sample_x, sample_y)
        
        # Now try to get parameters
        params = list(guide.parameters())
        print(f"Forced initialization: {len(params)} parameters")
        return len(params) > 0
    except Exception as e:
        print(f"Force initialization failed: {e}")
        return False


    print(f"Model parameters: {len(list(model.parameters()))}")
    print("Model parameter names:")
    for name, param in model.named_parameters():
        print(f"  {name}: {param.shape}")
    
    # Check if model forward actually has pyro.sample
    import inspect
    forward_source = inspect.getsource(model.forward)
    pyro_samples = forward_source.count('pyro.sample')
    print(f"Number of 'pyro.sample' calls in forward: {pyro_samples}")

In [ ]:
# ==============================================================================
# PYRO MODELS
# ==============================================================================

class SimplePyroModel(PyroModule):
    """Simple Bayesian neural network for comparison."""
    
    def __init__(self, input_dim, output_dim=283):
        super().__init__()
        
        self.network = PyroModule[nn.Sequential](
            PyroModule[nn.Linear](input_dim, config.PYRO_HIDDEN_DIMS[0]),
            PyroModule[nn.ReLU](),
            PyroModule[nn.Dropout](0.1),
            PyroModule[nn.Linear](config.PYRO_HIDDEN_DIMS[0], config.PYRO_HIDDEN_DIMS[1]),
            PyroModule[nn.ReLU](),
            PyroModule[nn.Dropout](0.1),
            PyroModule[nn.Linear](config.PYRO_HIDDEN_DIMS[1], output_dim)
        )
        
        # Set simple priors
        for name, param in self.network.named_parameters():
            if 'weight' in name:
                setattr(self.network, name,
                       PyroSample(dist.Normal(0., 0.5).expand(param.shape).to_event(param.dim())))
            elif 'bias' in name:
                setattr(self.network, name,
                       PyroSample(dist.Normal(0., 0.1).expand(param.shape).to_event(param.dim())))

    def forward(self, x, y=None):
        mu = self.network(x)
        
        # Simple noise model
        fgs_noise = pyro.sample("fgs_noise", dist.LogNormal(-6., 0.3))
        airs_noise = pyro.sample("airs_noise", dist.LogNormal(-5., 0.3))
        
        noise_tensor = torch.zeros(x.shape[0], mu.shape[1], device=x.device)
        noise_tensor[:, 0] = fgs_noise
        noise_tensor[:, 1:] = airs_noise
        
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mu, noise_tensor).to_event(1), obs=y)
        
        return mu

In [ ]:
class PhysicsInformedPyroModel(PyroModule):
    """Physics-informed model with sophisticated noise modeling."""
    
    def __init__(self, input_dim, output_dim=283, feature_names=None, wavelengths=None):
        super().__init__()
        
        self.feature_names = feature_names or []
        self.wavelengths = wavelengths
        self.output_dim = output_dim
        
        # Identify feature types
        self.stellar_indices = self._find_feature_indices(['Rs', 'Ts', 'Ms', 'log_g'])
        self.transit_indices = self._find_feature_indices(['fgs_slice', 'fgs_transit', 'fgs_snr'])
        self.physics_indices = self._find_feature_indices(['stellar_density', 'equilibrium_temp'])
        
        # Build networks
        self.stellar_net = self._build_network(len(self.stellar_indices), 32, 16) if self.stellar_indices else None
        self.transit_net = self._build_network(len(self.transit_indices), 64, 32) if self.transit_indices else None
        self.physics_net = self._build_network(len(self.physics_indices), 24, 12) if self.physics_indices else None
        
        # Remaining features
        remaining_dim = input_dim - len(self.stellar_indices + self.transit_indices + self.physics_indices)
        self.remaining_net = self._build_network(remaining_dim, 48, 24) if remaining_dim > 0 else None
        
        # Combiner
        combiner_input = sum([net.hidden_dim for net in [self.stellar_net, self.transit_net, 
                             self.physics_net, self.remaining_net] if net is not None])
        
        self.combiner = PyroModule[nn.Sequential](
            PyroModule[nn.Linear](combiner_input, 128),
            PyroModule[nn.ReLU](),
            PyroModule[nn.Dropout](0.15),
            PyroModule[nn.Linear](128, 64),
            PyroModule[nn.ReLU](),
            PyroModule[nn.Linear](64, output_dim)
        )
        
        self._set_physics_priors()
    
    def _find_feature_indices(self, keywords):
        """Find indices of features containing any of the keywords."""
        indices = []
        for i, name in enumerate(self.feature_names):
            if any(keyword.lower() in name.lower() for keyword in keywords):
                indices.append(i)
        return indices
    
    def _build_network(self, input_dim, hidden_dim, output_dim):
        """Build a simple network with stored hidden dimension."""
        if input_dim == 0:
            return None
        
        net = PyroModule[nn.Sequential](
            PyroModule[nn.Linear](input_dim, hidden_dim),
            PyroModule[nn.ReLU](),
            PyroModule[nn.Linear](hidden_dim, output_dim)
        )
        net.hidden_dim = output_dim  # Store for combiner calculation
        return net
    
    def _set_physics_priors(self):
        """Set physics-informed priors."""
        # Stellar: tight priors (well-understood physics)
        if self.stellar_net:
            self._set_network_priors(self.stellar_net, weight_scale=0.3, bias_scale=0.05)
        
        # Transit: medium priors
        if self.transit_net:
            self._set_network_priors(self.transit_net, weight_scale=0.5, bias_scale=0.1)
        
        # Physics-derived: tight priors
        if self.physics_net:
            self._set_network_priors(self.physics_net, weight_scale=0.4, bias_scale=0.08)
        
        # Remaining: wide priors
        if self.remaining_net:
            self._set_network_priors(self.remaining_net, weight_scale=0.7, bias_scale=0.15)
        
        # Combiner: balanced priors
        self._set_network_priors(self.combiner, weight_scale=0.5, bias_scale=0.1)
    
    def _set_network_priors(self, network, weight_scale, bias_scale):
        """Helper to set priors for a network."""
        for name, param in network.named_parameters():
            if 'weight' in name:
                setattr(network, name,
                       PyroSample(dist.Normal(0., weight_scale).expand(param.shape).to_event(param.dim())))
            elif 'bias' in name:
                setattr(network, name,
                       PyroSample(dist.Normal(0., bias_scale).expand(param.shape).to_event(param.dim())))

    def forward(self, x, y=None):
        batch_size = x.shape[0]
        processed_features = []
        
        # Process different feature types
        if self.stellar_net and self.stellar_indices:
            stellar_processed = self.stellar_net(x[:, self.stellar_indices])
            processed_features.append(stellar_processed)
        
        if self.transit_net and self.transit_indices:
            transit_processed = self.transit_net(x[:, self.transit_indices])
            processed_features.append(transit_processed)
        
        if self.physics_net and self.physics_indices:
            physics_processed = self.physics_net(x[:, self.physics_indices])
            processed_features.append(physics_processed)
        
        # Remaining features
        all_processed = self.stellar_indices + self.transit_indices + self.physics_indices
        remaining_indices = [i for i in range(x.shape[1]) if i not in all_processed]
        
        if self.remaining_net and remaining_indices:
            remaining_processed = self.remaining_net(x[:, remaining_indices])
            processed_features.append(remaining_processed)
        
        # Combine features
        if processed_features:
            combined = torch.cat(processed_features, dim=1)
        else:
            combined = x
        
        mu = self.combiner(combined)
        
        # Sample global noise parameters
        fgs_base_noise = pyro.sample("fgs_base_noise", 
                                    dist.LogNormal(torch.tensor(-6.0), torch.tensor(0.2)))
        airs_base_noise = pyro.sample("airs_base_noise", 
                                     dist.LogNormal(torch.tensor(-5.0), torch.tensor(0.2)))
        
        # Stellar temperature effects (if available)
        if self.stellar_indices and len(self.stellar_indices) >= 2:
            temp_effect = pyro.sample("temp_noise_effect", 
                                     dist.Normal(torch.tensor(0.0), torch.tensor(0.1)))
            stellar_temp = x[:, self.stellar_indices[1]]  # Assuming 2nd stellar feature is Ts
            norm_temp = (stellar_temp - 5500) / 1000
            temp_scaling = torch.exp(temp_effect * norm_temp)
            
            # Expand for batch
            fgs_noise = fgs_base_noise * temp_scaling
            airs_noise = airs_base_noise * temp_scaling
        else:
            # Expand scalars to match batch size
            fgs_noise = fgs_base_noise.expand(batch_size)
            airs_noise = airs_base_noise.expand(batch_size)
        
        # Wavelength-dependent scaling - make sure it's on correct device
        with pyro.plate("output_wavelengths", self.output_dim):
            wavelength_scaling = pyro.sample("wavelength_scaling",
                                           dist.LogNormal(torch.zeros(self.output_dim, device=x.device), 
                                                        torch.ones(self.output_dim, device=x.device) * 0.1))
        
        # Construct noise tensor - ensure proper broadcasting
        noise_tensor = torch.zeros(batch_size, self.output_dim, device=x.device)
        
        # FGS noise (first channel)
        if isinstance(fgs_noise, torch.Tensor):
            noise_tensor[:, 0] = fgs_noise * wavelength_scaling[0]
        else:
            noise_tensor[:, 0] = fgs_base_noise * wavelength_scaling[0]
        
        # AIRS noise (remaining channels)
        for i in range(1, self.output_dim):
            if isinstance(airs_noise, torch.Tensor):
                noise_tensor[:, i] = airs_noise * wavelength_scaling[i]
            else:
                noise_tensor[:, i] = airs_base_noise * wavelength_scaling[i]
        
        # Ensure positive noise values
        noise_tensor = torch.clamp(noise_tensor, min=1e-6)
        
        # Likelihood with proper plate
        with pyro.plate("data", batch_size):
            obs = pyro.sample("obs", 
                             dist.Normal(mu, noise_tensor).to_event(1), 
                             obs=y)
        
        return mu

def train_pyro_model(X_train, y_train, X_val, y_val, feature_names=None, wavelengths=None):
    """Train Pyro model with GPU support."""
    
    print(f"\n--- Training {config.MODEL_TYPE.upper()} model ---")
    
    # Convert to tensors and move to GPU
    X_train_tensor = torch.FloatTensor(X_train.values if hasattr(X_train, 'values') else X_train).to(device)
    y_train_tensor = torch.FloatTensor(y_train).to(device)
    X_val_tensor = torch.FloatTensor(X_val.values if hasattr(X_val, 'values') else X_val).to(device)
    y_val_tensor = torch.FloatTensor(y_val).to(device)
    
    # Clear parameter store
    pyro.clear_param_store()
    
    # Initialize model
    if config.MODEL_TYPE == 'physics_pyro':
        model = PhysicsInformedPyroModel(
            X_train_tensor.shape[1], 
            y_train_tensor.shape[1],
            feature_names=feature_names,
            wavelengths=wavelengths
        ).to(device)
    else:  # simple_pyro
        model = SimplePyroModel(
            X_train_tensor.shape[1],
            y_train_tensor.shape[1]
        ).to(device)
    
    print("Model created successfully")
    
    # Prepare sample data for initialization
    sample_x = X_train_tensor[:2].to(device)
    sample_y = y_train_tensor[:2].to(device)
    
    # Try different guide approaches
    guide = None
    svi = None
    setup_type = None
    
    print("=== Trying AutoMultivariateNormal ===")
    try:
        guide = AutoMultivariateNormal(model)
        
        # Initialize guide by running both model and guide
        with torch.no_grad():
            model(sample_x, sample_y)
            guide(sample_x, sample_y)
        
        guide_params = list(guide.parameters())
        print(f"Guide parameters: {len(guide_params)}")
        
        if len(guide_params) > 0:
            optimizer = PyroAdam({"lr": config.PYRO_LR})
            svi = SVI(model, guide, optimizer, loss=Trace_ELBO())
            
            # Test SVI step
            test_loss = svi.step(sample_x, sample_y)
            print(f"✅ AutoMultivariateNormal SUCCESS! Test loss: {test_loss:.4f}")
            # Watch
            # with pyro.poutine.trace() as tr:
            latent = guide.median(sample_x, sample_y)
            print("latent: ", latent)
            setup_type = "AutoMultivariateNormal"
        else:
            raise ValueError("No guide parameters created")
            
    except Exception as e:
        print(f"❌ AutoMultivariateNormal failed: {e}")
        
        print("=== Trying AutoDelta ===")
        try: 
            pyro.clear_param_store()
            guide = AutoDelta(model)
            
            with torch.no_grad():
                model(sample_x, sample_y)
                guide(sample_x, sample_y)
            
            guide_params = list(guide.parameters())
            print(f"AutoDelta parameters: {len(guide_params)}")
            
            if len(guide_params) > 0:
                optimizer = PyroAdam({"lr": config.PYRO_LR})
                svi = SVI(model, guide, optimizer, loss=Trace_ELBO())
                
                test_loss = svi.step(sample_x, sample_y)
                print(f"✅ AutoDelta SUCCESS! Test loss: {test_loss:.4f}")
                setup_type = "AutoDelta"
            else:
                raise ValueError("No AutoDelta parameters created")
                
        except Exception as e2:
            print(f"❌ AutoDelta failed: {e2}")
            
            print("=== Falling back to PyTorch training ===")
            # For PyTorch fallback, we need to modify the model's forward method
            # to not use pyro.sample statements for regular training
            print("❌ Pyro guides failed. You need to either:")
            print("1. Fix the pyro.sample statements in your model")
            print("2. Use a different training approach")
            print("3. Check that your model's forward method is compatible with Pyro")
            
            return None, None, None
    
    if svi is None:
        print("❌ Failed to create SVI - cannot proceed with training")
        return None, None, None
    
    print(f"\n🎉 Setup complete using: {setup_type}")
    
    # Training loop with batching for GPU memory management
    print(f"Training for {config.PYRO_EPOCHS} epochs...")
    losses = []
    
    for epoch in range(config.PYRO_EPOCHS):
        # Simple batch processing
        if X_train_tensor.shape[0] > config.PYRO_BATCH_SIZE:
            # Random batch for stochastic training
            batch_idx = torch.randperm(X_train_tensor.shape[0])[:config.PYRO_BATCH_SIZE]
            X_batch = X_train_tensor[batch_idx]
            y_batch = y_train_tensor[batch_idx]
        else:
            X_batch = X_train_tensor
            y_batch = y_train_tensor
        
        loss = svi.step(X_batch, y_batch)
        losses.append(loss)
        
        if epoch % 100 == 0:
            print(f"  Epoch {epoch:4d}, Loss: {loss:8.2f}")
            if config.USE_GPU:
                print(f"  GPU Memory: {torch.cuda.memory_allocated(device) / 1e9:.1f} GB")
    
    # Validation predictions
    print("Generating validation predictions...")
    try:
        predictive = Predictive(model, guide=guide, num_samples=config.PYRO_SAMPLES)
        
        with torch.no_grad():
            model.eval()
            guide.eval()
            
            trace = pyro.poutine.trace(guide).get_trace(X_val_tensor[:1])
            trace.compute_log_prob()
            val_predictions = predictive.get_samples(X_val_tensor)
            print('preds:', val_predictions.keys())  # Should include 'obs' and the latent sites
            pred_samples = val_predictions['obs']
            print('pred_samples: ', pred_samples)
            print(f"Sample shape: {pred_samples.shape}")
            print(f"Sample stats: min={pred_samples.min()}, max={pred_samples.max()}, mean={pred_samples.mean()}")

            mean_preds = []
            for _ in range(config.PYRO_SAMPLES):
                guide_trace = pyro.poutine.trace(guide).get_trace(X_val_tensor)
                model_trace = pyro.poutine.trace(pyro.poutine.replay(model, guide_trace)).get_trace(X_val_tensor)
                mean_preds.append(model_trace.nodes['obs']['value'].cpu())
        
            pred_samples_cpu = torch.stack(mean_preds)
            # Move back to CPU for numpy operations
            # pred_samples_cpu = pred_samples.cpu()
            
            val_quantile_preds = {
                0.05: torch.quantile(pred_samples_cpu, 0.05, dim=0).numpy(),
                0.50: torch.quantile(pred_samples_cpu, 0.50, dim=0).numpy(),
                0.95: torch.quantile(pred_samples_cpu, 0.95, dim=0).numpy()
            }
        model.train()
        guide.train()
    except Exception as e:
        print(f"⚠️ Prediction generation failed: {e}")
        print("Using model mean predictions instead...")
        
        with torch.no_grad():
            val_mean_pred = model(X_val_tensor).cpu().numpy()
            val_quantile_preds = {
                0.05: val_mean_pred * 0.95,  # Simple approximation
                0.50: val_mean_pred,
                0.95: val_mean_pred * 1.05
            }
    
    print(f"Training complete. Final loss: {losses[-1]:.2f}")
    return model.cpu(), guide, val_quantile_preds  # Move model back to CPU for saving



In [ ]:
# ==============================================================================
# COMPLETE PYRO PIPELINE: Ariel Data Challenge 2025 
# Features: Simple/Physics-Informed Pyro models, GPU support, submission handling
# ==============================================================================

import time
import pickle
import numpy as np
import pandas as pd
import polars as pl
import functools
import os
from tqdm import tqdm
import multiprocessing

# Pyro and PyTorch imports
import torch
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
from pyro.infer import SVI, Trace_ELBO, Predictive
from pyro.infer.autoguide import AutoMultivariateNormal, AutoDiagonalNormal
import torch.nn as nn

# Traditional ML imports (fallback)
import scipy.stats
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.multioutput import MultiOutputRegressor
import pywt

class Config:
    """Enhanced configuration with Pyro and GPU support."""
    
    # Paths
    DATA_PATH = '/kaggle/input/ariel-data-challenge-2025/'
    PREPROCESSED_PATH = '/kaggle/input/ariel-data-challenge-2025-af-npy/'
    OUTPUT_PATH = '/kaggle/working/ariel-data-2025-27'
    
    TRAIN_LABELS_PATH = os.path.join(DATA_PATH, 'train.csv')
    TRAIN_STAR_INFO_PATH = os.path.join(DATA_PATH, 'train_star_info.csv')
    TEST_STAR_INFO_PATH = os.path.join(DATA_PATH, 'test_star_info.csv')
    SAMPLE_SUBMISSION_PATH = os.path.join(DATA_PATH, 'sample_submission.csv')
    WAVELENGTHS_PATH = os.path.join(DATA_PATH, 'wavelengths.csv')
    A_RAW_PATH = os.path.join(PREPROCESSED_PATH, "a_raw_train.npy")
    F_RAW_PATH = os.path.join(PREPROCESSED_PATH, "f_raw_train.npy")
    
    # Model selection
    MODEL_TYPE = 'physics_pyro'  # Options: 'xgboost', 'simple_pyro', 'physics_pyro'
    
    # GPU Configuration
    USE_GPU = False
    GPU_DEVICE = 'cuda:0'  # Change to 'cuda:1' for second GPU if needed
    
    # Pyro hyperparameters
    PYRO_EPOCHS = 1000
    PYRO_LR = 0.005
    PYRO_BATCH_SIZE = 32  # For GPU memory management
    PYRO_SAMPLES = 200
    PYRO_HIDDEN_DIMS = [128, 64, 32]
    
    # Physics model options
    USE_WAVELENGTH_PHYSICS = True
    ADD_DERIVED_PHYSICS_FEATURES = True
    
    # Traditional parameters (fallback)
    VALIDATION_SPLIT = 0.1
    RANDOM_STATE = 42
    QUANTILES = [0.05, 0.50, 0.95]
    XGB_PARAMS = {
        'n_estimators': 400,
        'learning_rate': 0.04,
        'max_depth': 6,
        'subsample': 0.8,
        'colsample_bytree': 0.7,
        'random_state': 42,
        'tree_method': 'hist',
    }

config = Config()

# GPU Setup
def setup_gpu():
    """Setup GPU configuration for PyTorch and Pyro."""
    if config.USE_GPU and torch.cuda.is_available():
        device = torch.device(config.GPU_DEVICE)
        print(f"Using GPU: {torch.cuda.get_device_name(device)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(device).total_memory / 1e9:.1f} GB")
        
        # Optimize for GPU
        torch.backends.cudnn.benchmark = True
        torch.cuda.empty_cache()
        
        return device
    else:
        print("Using CPU")
        return torch.device('cpu')

device = setup_gpu()

# ==============================================================================
# DATA PREPROCESSING (Your existing functions)
# ==============================================================================

def f_read_and_preprocess(dataset, planet_ids):
    """Read the FGS1 files for all planet_ids and extract the time series."""
    print(f"Preprocessing FGS1 data for {dataset} set...")
    f_raw_data = np.full((len(planet_ids), 67500), np.nan, dtype=np.float32)
    for i, planet_id in tqdm(list(enumerate(planet_ids)), desc="FGS1"):
        path = f'/kaggle/input/ariel-data-challenge-2025/{dataset}/{int(planet_id)}/FGS1_signal_0.parquet'
        f_signal = pl.read_parquet(path)
        mean_signal = f_signal.cast(pl.Int32).sum_horizontal().cast(pl.Float32).to_numpy() / 1024
        net_signal = mean_signal[1::2] - mean_signal[0::2]
        f_raw_data[i] = net_signal
    return f_raw_data

def a_read_and_preprocess(dataset, planet_ids):
    """Read the AIRS-CH0 files for all planet_ids and extract the time series."""
    print(f"Preprocessing AIRS-CH0 data for {dataset} set...")
    a_raw_data = np.full((len(planet_ids), 5625), np.nan, dtype=np.float32)
    for i, planet_id in tqdm(list(enumerate(planet_ids)), desc="AIRS-CH0"):
        path = f'/kaggle/input/ariel-data-challenge-2025/{dataset}/{int(planet_id)}/AIRS-CH0_signal_0.parquet'
        signal = pl.read_parquet(path)
        mean_signal = signal.cast(pl.Int32).sum_horizontal().cast(pl.Float32).to_numpy() / (32*356)
        net_signal = mean_signal[1::2] - mean_signal[0::2]
        a_raw_data[i] = net_signal
    return a_raw_data

def official_competition_score(y_true, y_pred, sigma_pred, naive_mean, naive_sigma,
                               fsg_sigma_true=1e-6, airs_sigma_true=1e-5, fgs_weight=2.0):
    """Official weighted Gaussian Log-Likelihood metric."""
    y_true, y_pred, sigma_pred = np.array(y_true), np.array(y_pred), np.array(sigma_pred)
    sigma_pred = np.clip(sigma_pred, 1e-15, None)

    sigma_true_per_channel = np.append(np.array([fsg_sigma_true]), np.ones(y_true.shape[1] - 1) * airs_sigma_true)
    sigma_true = np.tile(sigma_true_per_channel, (y_true.shape[0], 1))

    GLL_pred = scipy.stats.norm.logpdf(y_true, loc=y_pred, scale=sigma_pred)
    GLL_true = scipy.stats.norm.logpdf(y_true, loc=y_true, scale=sigma_true)
    GLL_mean = scipy.stats.norm.logpdf(y_true, loc=naive_mean, scale=naive_sigma)
    
    denominator = GLL_true - GLL_mean
    ind_scores = (GLL_pred - GLL_mean) / (denominator + 1e-9)

    weights_per_channel = np.append(np.array([fgs_weight]), np.ones(y_true.shape[1] - 1))
    weights = np.tile(weights_per_channel, (y_true.shape[0], 1))

    final_score = np.average(ind_scores, weights=weights)
    return float(np.clip(final_score, 0.0, 1.0))

def maximal_feature_engineering(f_raw, a_raw, star_info_df):
    """Your existing feature engineering function."""
    print("Engineering features...")
    
    fgs_pre = f_raw[:, :20500]; fgs_post = f_raw[:, 47000:]
    fgs_unobscured_mean = (fgs_pre.mean(axis=1) + fgs_post.mean(axis=1)) / 2
    fgs_unobscured_std = (fgs_pre.std(axis=1) + fgs_post.std(axis=1)) / 2
    fgs_transit = f_raw[:, 23500:44000]
    
    features = {}
    for i in range(5):
        f_slice_mean = fgs_transit[:, i*4100:(i+1)*4100].mean(axis=1)
        features[f'fgs_slice_{i+1}'] = (fgs_unobscured_mean - f_slice_mean) / fgs_unobscured_mean
    
    features['fgs_transit_std'] = fgs_transit.std(axis=1)
    features['fgs_transit_skew'] = scipy.stats.skew(fgs_transit, axis=1)
    features['fgs_transit_kurtosis'] = scipy.stats.kurtosis(fgs_transit, axis=1)
    features['fgs_snr'] = (fgs_unobscured_mean - fgs_transit.mean(axis=1)) / fgs_unobscured_std
    features_df = pd.DataFrame(features, index=star_info_df.index)

    fft_coeffs = np.fft.fft(fgs_transit, axis=1)
    for i in range(1, 6):
        features_df[f'fgs_fft_mag_{i}'] = np.abs(fft_coeffs[:, i])
    for level in range(1, 4):
        coeffs = pywt.wavedec(fgs_transit, 'db4', level=level, axis=1)
        features_df[f'fgs_wavelet_std_level{level}'] = np.std(coeffs[0], axis=1)
        features_df[f'fgs_wavelet_mean_level{level}'] = np.mean(coeffs[0], axis=1)

    meta_df = star_info_df.copy().fillna(star_info_df.median())
    meta_df['log_g_proxy'] = np.log1p(meta_df['Ms']) - 2 * np.log1p(meta_df['Rs'])
    meta_df['rho_star_proxy'] = meta_df['Ms'] / (meta_df['Rs']**3)
    
    poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
    poly_cols = ['Rs', 'Ts', 'Mp', 'P']
    poly_features = poly.fit_transform(meta_df[poly_cols])
    poly_df = pd.DataFrame(poly_features, columns=poly.get_feature_names_out(poly_cols), index=meta_df.index)
    
    final_features_df = pd.concat([features_df, meta_df, poly_df], axis=1)
    final_features_df = final_features_df.loc[:, ~final_features_df.columns.duplicated()]

    if config.ADD_DERIVED_PHYSICS_FEATURES:
        final_features_df = add_derived_physics_features(final_features_df, meta_df)

    print(f"Created {final_features_df.shape[1]} features in total.")
    return final_features_df.fillna(0)

def add_derived_physics_features(features_df, star_info_df):
    """Add physics-derived features."""
    enhanced_df = features_df.copy()
    
    if 'Rs' in star_info_df.columns and 'Ms' in star_info_df.columns:
        enhanced_df['stellar_density'] = star_info_df['Ms'] / (star_info_df['Rs'] ** 3)
        enhanced_df['log_g_calculated'] = np.log10(star_info_df['Ms'] / (star_info_df['Rs'] ** 2))
    
    if 'Mp' in star_info_df.columns and 'P' in star_info_df.columns:
        enhanced_df['semi_major_axis'] = ((star_info_df['P'] / 365.25) ** (2/3) * 
                                        star_info_df['Ms'] ** (1/3))
        
        if 'Ts' in star_info_df.columns and 'Rs' in star_info_df.columns:
            enhanced_df['equilibrium_temp'] = (star_info_df['Ts'] * 
                                             np.sqrt(star_info_df['Rs'] / 
                                                   (2 * enhanced_df['semi_major_axis'])))
    
    return enhanced_df

def train_xgboost_quantiles(X_train, y_train, X_val, y_val):
    """Fallback XGBoost training."""
    print("\\n--- Training XGBoost Quantile Models ---")
    
    val_quantile_preds = {}
    for q in config.QUANTILES:
        print(f"Training quantile: {q}")
        model = xgb.XGBRegressor(**config.XGB_PARAMS, objective='reg:quantileerror', quantile_alpha=q)
        wrapper = MultiOutputRegressor(model, n_jobs=-1)
        wrapper.fit(X_train, y_train)
        val_quantile_preds[q] = wrapper.predict(X_val)
    
    return val_quantile_preds

# ==============================================================================
# SUBMISSION FUNCTIONS
# ==============================================================================

def make_pyro_predictions(model, guide, X_test, feature_names=None, wavelengths=None):
    """Make predictions with trained Pyro model."""
    
    print("Making Pyro predictions...")
    
    # Recreate model architecture for loading
    if config.MODEL_TYPE == 'physics_pyro':
        model_for_prediction = PhysicsInformedPyroModel(
            X_test.shape[1], 283,
            feature_names=feature_names,
            wavelengths=wavelengths
        ).to(device)
    else:
        model_for_prediction = SimplePyroModel(X_test.shape[1], 283).to(device)
    
    # Load state if model is provided as path
    if isinstance(model, str):
        model_for_prediction.load_state_dict(torch.load(model, map_location=device))
        guide.load_state_dict(torch.load(model.replace('model', 'guide'), map_location=device))
        model = model_for_prediction
    else:
        model = model.to(device)
    
    X_test_tensor = torch.FloatTensor(X_test.values if hasattr(X_test, 'values') else X_test).to(device)
    
    predictive = Predictive(model, guide=guide, num_samples=config.PYRO_SAMPLES)
    
    with torch.no_grad():
        # Process in batches to manage GPU memory
        all_predictions = []
        batch_size = min(config.PYRO_BATCH_SIZE, X_test_tensor.shape[0])
        
        for i in range(0, X_test_tensor.shape[0], batch_size):
            X_batch = X_test_tensor[i:i+batch_size]
            batch_predictions = predictive(X_batch)
            all_predictions.append(batch_predictions['obs'].cpu())
        
        # Combine all predictions
        pred_samples = torch.cat(all_predictions, dim=1)
        
        test_quantile_preds = {
            0.05: torch.quantile(pred_samples, 0.05, dim=0).numpy(),
            0.50: torch.quantile(pred_samples, 0.50, dim=0).numpy(),
            0.95: torch.quantile(pred_samples, 0.95, dim=0).numpy()
        }
    
    return test_quantile_preds

def save_models(model, guide, feature_names, output_path):
    """Save models and metadata."""
    os.makedirs(output_path, exist_ok=True)
    
    if config.MODEL_TYPE in ['simple_pyro', 'physics_pyro']:
        torch.save(model.state_dict(), os.path.join(output_path, 'pyro_model.pth'))
        torch.save(guide.state_dict(), os.path.join(output_path, 'pyro_guide.pth'))
        
        # Save model metadata
        metadata = {
            'model_type': config.MODEL_TYPE,
            'feature_names': feature_names,
            'input_dim': len(feature_names),
            'output_dim': 283
        }
        
        with open(os.path.join(output_path, 'model_metadata.pkl'), 'wb') as f:
            pickle.dump(metadata, f)
        
        print(f"Pyro models saved to: {output_path}")
    
    # Always save feature columns for compatibility
    with open(os.path.join(output_path, 'feature_columns.pkl'), 'wb') as f:
        pickle.dump(feature_names, f)

def load_models(output_path):
    """Load saved models and metadata."""
    
    # Load metadata
    with open(os.path.join(output_path, 'model_metadata.pkl'), 'rb') as f:
        metadata = pickle.load(f)
    
    with open(os.path.join(output_path, 'feature_columns.pkl'), 'rb') as f:
        feature_names = pickle.load(f)
    
    # Load appropriate model
    if metadata['model_type'] == 'physics_pyro':
        model = PhysicsInformedPyroModel(
            metadata['input_dim'], 
            metadata['output_dim'],
            feature_names=metadata['feature_names']
        )
    elif metadata['model_type'] == 'simple_pyro':
        model = SimplePyroModel(
            metadata['input_dim'],
            metadata['output_dim']
        )
    else:
        raise ValueError(f"Unknown model type: {metadata['model_type']}")
    
    # Load model states
    model.load_state_dict(torch.load(os.path.join(output_path, 'pyro_model.pth'), map_location='cpu'))
    
    guide = AutoMultivariateNormal(model)
    guide.load_state_dict(torch.load(os.path.join(output_path, 'pyro_guide.pth'), map_location='cpu'))
    
    return model, guide, feature_names, metadata

# ==============================================================================
# MAIN EXECUTION PIPELINE
# ==============================================================================

def main():
    """Main execution pipeline with model switching."""
    
    print(f"\\nUsing model: {config.MODEL_TYPE}")
    print(f"Device: {device}")
    if config.USE_GPU:
        print(f"GPU: {torch.cuda.get_device_name(device)}")
    
    is_submission_run = False # os.path.exists(config.TEST_STAR_INFO_PATH)

    if is_submission_run:
        # --- SUBMISSION MODE ---
        print("\\n" + "="*60)
        print("======         SUBMISSION MODE          ======")
        print("="*60 + "\\n")

        # Load test data
        sample_submission = pd.read_csv(config.SAMPLE_SUBMISSION_PATH, index_col='planet_id')
        test_star_info_df = pd.read_csv(config.TEST_STAR_INFO_PATH, index_col='planet_id')
        wavelengths_df = pd.read_csv(config.WAVELENGTHS_PATH)
        target_column_names = wavelengths_df.columns.tolist()

        # Process test data
        f_raw_test = f_read_and_preprocess('test', test_star_info_df.index)
        a_raw_test = a_read_and_preprocess('test', test_star_info_df.index)
        test_features_df = maximal_feature_engineering(f_raw_test, a_raw_test, test_star_info_df)
        
        # Load models and make predictions
        if config.MODEL_TYPE in ['simple_pyro', 'physics_pyro']:
            try:
                model, guide, feature_names, metadata = load_models(config.OUTPUT_PATH)
                test_features_df = test_features_df[feature_names]
                
                # Get wavelengths for physics model
                wavelengths = None
                if config.MODEL_TYPE == 'physics_pyro' and config.USE_WAVELENGTH_PHYSICS:
                    try:
                        # Extract wavelengths from column names
                        wavelengths = []
                        for col in target_column_names:
                            if col.startswith('wl_'):
                                try:
                                    wl_val = float(col.replace('wl_', ''))
                                    wavelengths.append(wl_val)
                                except:
                                    wavelengths.append(len(wavelengths) + 1)
                    except:
                        wavelengths = None
                
                test_quantile_preds = make_pyro_predictions(
                    model, guide, test_features_df, feature_names, wavelengths
                )
                
            except Exception as e:
                print(f"Error loading Pyro model: {e}")
                print("Falling back to XGBoost...")
                config.MODEL_TYPE = 'xgboost'
        
        if config.MODEL_TYPE == 'xgboost':
            # Load XGBoost models (your existing code)
            with open(os.path.join(config.OUTPUT_PATH, 'feature_columns.pkl'), 'rb') as f:
                train_cols = pickle.load(f)
            
            test_features_df = test_features_df[train_cols]
            
            trained_models = {}
            for q in config.QUANTILES:
                model_path = os.path.join(config.OUTPUT_PATH, f'model_quantile_{q}.pkl')
                with open(model_path, 'rb') as f:
                    trained_models[q] = pickle.load(f)
            
            test_quantile_preds = {}
            for q in config.QUANTILES:
                test_quantile_preds[q] = trained_models[q].predict(test_features_df)

        # Create submission
        y_pred_test = test_quantile_preds[0.50].clip(0, None)
        lower_test, upper_test = test_quantile_preds[0.05], test_quantile_preds[0.95]
        
        sigma_raw_test = (upper_test - lower_test) / 3.29
        sigma_raw_test[sigma_raw_test < 0] = 1e-10
        
        # For Pyro models, uncertainties are better calibrated, but we can still apply minimal scaling
        if config.MODEL_TYPE in ['simple_pyro', 'physics_pyro']:
            sigma_pred_test = sigma_raw_test * 1.1  # Minimal adjustment for Pyro
        else:
            # Load calibration for XGBoost
            try:
                with open(os.path.join(config.OUTPUT_PATH, 'calibration_params.pkl'), 'rb') as f:
                    calibration_params = pickle.load(f)
                sigma_pred_test = (sigma_raw_test * calibration_params['scaling']) + calibration_params['additive']
            except:
                sigma_pred_test = sigma_raw_test * 1.2  # Default scaling
        
        # Format submission
        pred_df = pd.DataFrame(y_pred_test, index=sample_submission.index, columns=target_column_names)
        sigma_df = pd.DataFrame(sigma_pred_test, index=sample_submission.index, 
                               columns=[f"sigma_{i+1}" for i in range(len(target_column_names))])
        submission_df = pd.concat([pred_df, sigma_df], axis=1)
        
        submission_df.to_csv('submission.csv')
        print("\\n'submission.csv' created successfully!")
        print(f"\\nSubmission preview ({config.MODEL_TYPE}):")
        print(submission_df.head())

    else:
        # --- TRAINING MODE ---
        print("\\n" + "="*60)
        print("======          TRAINING MODE           ======")
        print("="*60 + "\\n")
        
        # Load training data
        train_labels_df = pd.read_csv(config.TRAIN_LABELS_PATH, index_col='planet_id')
        train_star_info_df = pd.read_csv(config.TRAIN_STAR_INFO_PATH, index_col='planet_id').loc[train_labels_df.index]
        
        print("Loading pre-processed training data...")
        f_raw_train, a_raw_train = np.load(config.F_RAW_PATH), np.load(config.A_RAW_PATH)
        
        train_features_df = maximal_feature_engineering(f_raw_train, a_raw_train, train_star_info_df)
        train_labels = train_labels_df.values
        naive_mu_train, naive_sigma_train = np.mean(train_labels), np.std(train_labels)

        # Train-validation split
        X_train, X_val, y_train, y_val = train_test_split(
            train_features_df, train_labels, 
            test_size=config.VALIDATION_SPLIT, 
            random_state=config.RANDOM_STATE
        )
        
        feature_names = train_features_df.columns.tolist()
        
        # Get wavelengths for physics model
        wavelengths = None
        if config.MODEL_TYPE == 'physics_pyro' and config.USE_WAVELENGTH_PHYSICS:
            try:
                wavelengths_df = pd.read_csv(config.WAVELENGTHS_PATH)
                wavelengths = []
                for col in wavelengths_df.columns:
                    if col.startswith('wl_'):
                        try:
                            wl_val = float(col.replace('wl_', ''))
                            wavelengths.append(wl_val)
                        except:
                            wavelengths.append(len(wavelengths) + 1)
            except:
                print("Could not load wavelengths, using default indexing")
                wavelengths = None

        # Train model based on selection
        if config.MODEL_TYPE in ['simple_pyro', 'physics_pyro']:
            # Train Pyro model
            model, guide, val_quantile_preds = train_pyro_model(
                X_train, y_train, X_val, y_val, feature_names, wavelengths
            )
            
            # Evaluate
            y_pred_val = val_quantile_preds[0.50]
            lower_val, upper_val = val_quantile_preds[0.05], val_quantile_preds[0.95]
            sigma_raw_val = (upper_val - lower_val) / 3.29
            sigma_raw_val[sigma_raw_val < 0] = 1e-10
            
            # Minimal calibration for Pyro (they're usually well-calibrated)
            sigma_val = sigma_raw_val * 1.1
            
            pyro_score = official_competition_score(
                y_val, y_pred_val, sigma_val, naive_mu_train, naive_sigma_train
            )
            
            print(f"\\n{config.MODEL_TYPE.upper()} Validation Score: {pyro_score:.4f}")
            
            # Save models
            save_models(model, guide, feature_names, config.OUTPUT_PATH)
            # First model's prediction and evaluation
            y_pred_val = val_quantile_preds[0.50]
            lower_val, upper_val = val_quantile_preds[0.05], val_quantile_preds[0.95]
            sigma_raw_val = (upper_val - lower_val) / 3.29
            sigma_raw_val[sigma_raw_val < 0] = 1e-10
            sigma_val = sigma_raw_val * 1.1
            
            print("DIAGNOSTICS AS PER CHAT 1:")
            abs_err = np.abs(y_val - y_pred_val)
            normed_err = abs_err / sigma_val
            print("Mean absolute error:", abs_err.mean())
            print("Mean normalized error:", normed_err.mean())
            print("Max normalized error:", normed_err.max())
            print("Min normalized error:", normed_err.min())
            print("END DIAGNOSTICS AS PER CHAT 1")

            import matplotlib.pyplot as plt

            plt.scatter(y_pred_val, sigma_val, alpha=0.5)
            plt.xlabel("Prediction")
            plt.ylabel("Predicted σ")
            plt.title("Prediction vs Uncertainty")
            plt.grid(True)
            plt.show()

            
            pyro_score = official_competition_score(
                y_val, y_pred_val, sigma_val, naive_mu_train, naive_sigma_train
            )
            
            print(f"\nPYRO Validation Score: {pyro_score:.4f}")
            print("DIAGNOSTICS AS PER CHAT 2:")
            print("Mean pred:", y_pred_val.mean().item())
            print("Std pred:", y_pred_val.std().item())
            print("Any NaN?", np.isnan(y_pred_val).any())
            print("y_val mean/std:", y_val.mean().item(), y_val.std().item())
            print("Pred mean/std:", y_pred_val.mean().item(), y_pred_val.std().item())
            print("Sigma mean:", sigma_val.mean().item())
            print(y_pred_val[:5])
            print(y_val[:5])
            print("END DIAGNOSTICS AS PER CHAT 2")

            # Retrain on full dataset
            print("\\nRetraining on full dataset...")
            full_model, full_guide, val_quantile_preds = train_pyro_model(
                train_features_df, train_labels, 
                train_features_df[:100], train_labels[:100],  # Dummy validation for interface
                feature_names, wavelengths
            )
            
            # Save final models
            save_models(full_model, full_guide, feature_names, config.OUTPUT_PATH)
            
        else:  # XGBoost
            val_quantile_preds = train_xgboost_quantiles(X_train, y_train, X_val, y_val)
            
            # Your existing calibration logic
            y_pred_val, lower_val, upper_val = val_quantile_preds[0.50], val_quantile_preds[0.05], val_quantile_preds[0.95]
            sigma_raw_val = (upper_val - lower_val) / 3.29
            sigma_raw_val[sigma_raw_val < 0] = 1e-10

            print("\\nSearching for calibration factors...")
            best_score, best_scaling, best_additive = -1.0, 1.0, 0.0
            
            for scaling in [1.0, 1.2, 1.5, 2.0, 2.5]:
                for additive in [0.0, 0.0005, 0.001, 0.0015]:
                    sigma_calibrated = (sigma_raw_val * scaling) + additive
                    score = official_competition_score(y_val, y_pred_val, sigma_calibrated, naive_mu_train, naive_sigma_train)
                    
                    if score > best_score:
                        best_score, best_scaling, best_additive = score, scaling, additive
            
            print(f"Best XGBoost Score: {best_score:.4f} (Scale={best_scaling}, Add={best_additive})")
            
            # Save calibration and retrain models
            os.makedirs(config.OUTPUT_PATH, exist_ok=True)
            calibration_params = {'scaling': best_scaling, 'additive': best_additive}
            with open(os.path.join(config.OUTPUT_PATH, 'calibration_params.pkl'), 'wb') as f:
                pickle.dump(calibration_params, f)
            
            with open(os.path.join(config.OUTPUT_PATH, 'feature_columns.pkl'), 'wb') as f:
                pickle.dump(feature_names, f)

            print("\\nRetraining XGBoost on full dataset...")
            for q in config.QUANTILES:
                model = xgb.XGBRegressor(**config.XGB_PARAMS, objective='reg:quantileerror', quantile_alpha=q)
                wrapper = MultiOutputRegressor(model, n_jobs=-1)
                wrapper.fit(train_features_df, train_labels)
                
                model_path = os.path.join(config.OUTPUT_PATH, f'model_quantile_{q}.pkl')
                with open(model_path, 'wb') as f:
                    pickle.dump(wrapper, f)
        
        print("\\nTraining complete!")

if __name__ == '__main__':
    
    # Clear GPU memory at start
    if config.USE_GPU and torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    try:
        main()
    finally:
        # Clean up GPU memory
        if config.USE_GPU and torch.cuda.is_available():
            torch.cuda.empty_cache()
            print(f"\\nFinal GPU memory usage: {torch.cuda.memory_allocated(device) / 1e9:.1f} GB")
